In [ ]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
data = ticker.history(period="3y")
data.drop(columns=["Volume", "Dividends", "Stock Splits"], inplace=True)
data = data.diff()[1:]
data.head()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])
fig.update_layout(title=f"Historic price differences for {ticker.ticker}",
                  yaxis_title='Price (USD)')
fig.show()

In [ ]:
import jpt.sequential_trees
variables = jpt.infer_from_dataframe(data, precision=0.1)
variables

In [ ]:
template_tree = jpt.trees.JPT(variables, min_samples_leaf=10)
model = jpt.sequential_trees.SequentialJPT(template_tree)
model.fit(sequences=[data])

In [ ]:
evidence = model.bind([{col: data[col][-1] for col in data.columns}] + [dict()] * 33)
mpes, likelihood = model.mpe(evidence)

opens = [mpe["Open"].upper for mpe in mpes]
highs = [mpe["High"].upper for mpe in mpes]
lows = [mpe["Low"].lower for mpe in mpes]
closes = [mpe["Close"].upper for mpe in mpes]

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=opens,
                high=highs,
                low=lows,
                close=closes)])
fig.update_layout(title=f"Predicted price differences for {ticker.ticker}",
                  yaxis_title='Price (USD)')
fig.show()